In [1]:
import pandas as pd
import numpy as np
import os

import datetime
HOME = "/home/dimitri/epita/big_data/project/bourse/data/"

In [ ]:
def extract_date_hours(path):
    file_name = path.split('/')[-1]  # Extract the file name
    date_str, time_str = file_name.split(' ')[1:3]  # Extract date and time parts
    time_str = ".".join(time_str.split('.')[0:2])
    date_time_str = f"{date_str} {time_str}"
    date_time = datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
    return date_time

def merge_df(file_path, df):
    df_tmp = pd.read_pickle(
    file_path,
    )
    # df.reset_index(drop=True, inplace=True)
    df_tmp['date'] = extract_date_hours(file_path)
    return pd.concat([df, df_tmp])
def extract_symbole(df):
    df["boursorama"] = df["symbol"].copy()
    df["symbol"] = df["symbol"].apply(lambda x: x[3: len(x)])
def extract_identifiant_companies(df):
    df["prefix"] = df["boursorama"].apply(lambda x: x[0:3])
    
def get_df_bourso(num_files=100):
    df = pd.DataFrame()
    dir = os.listdir(HOME + "boursorama")
    i = 0
    for dir_date in dir :
        list_file_path = os.listdir(HOME + "boursorama/" + dir_date)
        for file_path in list_file_path :
            df = merge_df(HOME + "boursorama/" + dir_date + "/" + file_path, df)
            if (i == num_files) :
                break
            i+=1
        break
    extract_symbole(df)
    extract_identifiant_companies(df)

    return df

In [3]:
import pandas as pd
import glob
import os
import re

def load_dataset(data_path, n):
    # Print the number of files in the directory
    files = os.listdir(data_path)
    print(f"Number of files in the directory: {len(files)}")

    # Use glob to get all files that start with 'Euronext_Equities_' and end with .csv or .xlsx
    file_pattern = os.path.join(data_path, "Euronext_Equities_*.*")
    files = glob.glob(file_pattern)

    # List to store individual dataframes
    df_list = []
    counter = 0

    # Loop through each file and process based on file extension
    for file in files:
        if counter == n:
            break
        try:
            if file.lower().endswith('.csv'):
                df = pd.read_csv(file, encoding='utf-8', sep='\t')
            elif file.lower().endswith('.xlsx'):
                df = pd.read_excel(file)
            else:
                # Skip unknown file types
                print("Unsupported file type")
                continue

            # Extract the date from the filename using a regex (assuming format YYYY-MM-DD)
            date_match = re.search(r'(\d{4}-\d{2}-\d{2})', os.path.basename(file))
            if date_match:
                file_date = date_match.group(1)
                # Add the second and milisecond to the date
                file_date = file_date + " 00:00:00.000000"
                file_date = datetime.datetime.strptime(file_date, '%Y-%m-%d %H:%M:%S.%f')
                # Add the date as a new column (as datetime type)
                df['date'] = file_date
            else:
                # Optionally log or handle files without a proper date in the filename
                df['file_date'] = pd.NaT

            # Append the dataframe to the list
            df_list.append(df)
            counter += 1
        except Exception as e:
            print(f"Error processing file {file}: {e}")

    return df_list

def get_df_euronext(n):
    # Define the path to the directory containing the files
    data_path = HOME + "euronext"

    df_list = load_dataset(data_path, n)

    # Concatenate all dataframes into one robust dataframe
    if df_list:
        combined_df = pd.concat(df_list, ignore_index=True)
    else:
        combined_df = pd.DataFrame()

    # Standardize column names: trim whitespace, lower case, and replace spaces with underscores
    combined_df.columns = combined_df.columns.str.strip().str.lower().str.replace(' ', '_')

    # Define a mapping of equivalent column names to merge differences between CSV and XLSX files
    column_synonyms = {
        'ticker': ['ticker', 'symbol'],
        'name': ['company', 'company_name', 'name'],
        'price': ['price', 'closing_price', 'last_price', 'last'],
        'currency': ['currency', 'trading_currency'],
        'open': ['open', 'open_price'],
        'high': ['high', 'high_price'],
        'low': ['low', 'low_price'],
        'last_trade_time': ['last_trade_time', 'last_trade_mic_time', 'last_date/time']
    }

    # Merge equivalent columns
    for canonical, synonyms in column_synonyms.items():
        # Find which of the synonym columns are present in the dataframe
        cols_present = [col for col in synonyms if col in combined_df.columns]
        if len(cols_present) > 1:
            # Merge the columns: use the first non-null value among the columns
            combined_df[canonical] = combined_df[cols_present[0]].combine_first(combined_df[cols_present[1]])
            for col in cols_present[2:]:
                combined_df[canonical] = combined_df[canonical].combine_first(combined_df[col])
            # Drop the extra synonym columns, keeping the canonical one
            for col in cols_present:
                if col != canonical:
                    combined_df.drop(columns=col, inplace=True)
        elif len(cols_present) == 1 and cols_present[0] != canonical:
            # Rename the column to the canonical name
            combined_df.rename(columns={cols_present[0]: canonical}, inplace=True)

    # Remove the closing price column if it exists
    if 'closing_price_datetime' in combined_df.columns:
        combined_df.drop(columns='closing_price_datetime', inplace=True)

    # Optionally, drop duplicates
    combined_df.drop_duplicates(inplace=True)

    # Remove the rows where the isin value is NaN
    combined_df = combined_df[~combined_df['isin'].isna()]

    # Add a new column 'pea' based on the currency column
    if 'currency' in combined_df.columns:
        combined_df['pea'] = combined_df['currency'].apply(
            lambda x: True if isinstance(x, str) and x.upper() == 'EUR' else False)
    else:
        combined_df['pea'] = False

    # Reset the index of the dataframe
    combined_df.reset_index(drop=True, inplace=True)

    #   Replace invalid values with NaN
    combined_df["high"] = combined_df["high"].replace('-', np.nan)
    combined_df["low"] = combined_df["low"].replace('-', np.nan)

    # Convert columns to float
    combined_df["high"] = combined_df["high"].astype(float)
    combined_df["low"] = combined_df["low"].astype(float)
    
    # Trier les données par 'isin' et 'date' pour garantir l'ordre correct
    if 'isin' in combined_df.columns and 'date' in combined_df.columns:
        combined_df.sort_values(by=['isin', 'date'], inplace=True)

    # Ajouter une colonne 'close' qui correspond à la valeur 'open' du jour suivant pour chaque 'isin'
    if 'open' in combined_df.columns:
        combined_df['close'] = combined_df.groupby('isin')['open'].shift(-1)
    else:
        combined_df['close'] = pd.NA

    # Réinitialiser l'index après avoir ajouté la colonne 'close'
    combined_df.reset_index(drop=True, inplace=True)

    return combined_df


In [65]:
df_boursorama = get_df_bourso(1)
df_boursorama.head(10)

,last,volume,symbol,name,date,boursorama,prefix
symbol,,,,,,,
1rPABBV,58.01,0,ABBV,ABBVIE,2019-02-28 13:32:01.512834,1rPABBV,1rP
1rPAC,37.29,405421,AC,ACCOR,2019-02-28 13:32:01.512834,1rPAC,1rP
1rPACNV,46.02,0,ACNV,ACCOR,2019-02-28 13:32:01.512834,1rPACNV,1rP
1rPADP,171.7,22674,ADP,ADP,2019-02-28 13:32:01.512834,1rPADP,1rP
1rPAF,10.86,3512926,AF,AIR FRANCE - KLM,2019-02-28 13:32:01.512834,1rPAF,1rP
1rPAI,109.65,163127,AI,AIR LIQUIDE,2019-02-28 13:32:01.512834,1rPAI,1rP
1rPAIR,113.24,285724,AIR,AIRBUS,2019-02-28 13:32:01.512834,1rPAIR,1rP
1rPAKA,54.8,2613,AKA,AKKA TECHNOLOGIES,2019-02-28 13:32:01.512834,1rPAKA,1rP
1rPALUNV,3.46,0,ALUNV,ALCATEL I15,2019-02-28 13:32:01.512834,1rPALUNV,1rP


In [155]:
df_euronext = get_df_euronext(2)
df_euronext.head()

Number of files in the directory: 775


,name,isin,ticker,market,currency,open,high,low,price,last_trade_time,time_zone,volume,turnover,date,pea,close
0,SCHLUMBERGER,AN8068571086,SLB,Euronext Paris,EUR,36.75,36.85,35.35,35.50,18/02/2022 17:35,CET,21155,758445.00,2022-02-18,True,38.20
1,SCHLUMBERGER,AN8068571086,SLB,Euronext Paris,EUR,38.20,38.25,36.85,37.75,31/03/2022 17:35,CET,32389,1213839.65,2022-03-31,True,NaN
2,SOLVAY,BE0003470755,SOLB,"Euronext Brussels, Paris",EUR,109.75,110.05,107.50,108.30,18/02/2022 17:38,CET,146619,15928350.45,2022-02-18,True,93.04
3,SOLVAY,BE0003470755,SOLB,"Euronext Brussels, Paris",EUR,93.04,94.62,89.32,89.46,31/03/2022 17:35,CET,354284,32030180.64,2022-03-31,True,NaN
4,MONTEA,BE0003853703,MONT,"Euronext Brussels, Paris",EUR,109.00,109.40,105.80,106.80,18/02/2022 17:35,CET,7743,829569.40,2022-02-18,True,117.60


In [121]:
def merge_dataset(df_boursorama, df_euronext, delete_name_alone=True):
    # delete_name_alone : delete the rows when the name is only in bourso and not in euronext
    if delete_name_alone:
        # Filter rows where the name exists in both datasets
        df_boursorama2 = df_boursorama[df_boursorama['name'].isin(df_euronext['name'])].copy()
    else:
        df_boursorama2 = df_boursorama.copy()

    # Get all unique columns from both datasets
    all_columns = list(set(df_euronext.columns).union(set(df_boursorama2.columns)))

    # Ensure both dataframes have the same columns
    for col in all_columns:
        if col not in df_euronext.columns:
            df_euronext[col] = np.nan
        if col not in df_boursorama2.columns:
            df_boursorama2[col] = np.nan

    # Concatenate the two datasets
    df = pd.concat([df_boursorama2, df_euronext], ignore_index=True)

    # Fill missing 'isin' values in df with the mapping from df_euronext
    isin_mapping = df_euronext.set_index('name')['isin'].to_dict()
    df['isin'] = df['isin'].fillna(df['name'].map(isin_mapping))

    # Remove duplicates based on all columns
    

    # Optionally, you can remove duplicates based on specific columns (e.g., 'name' and 'isin')
    # df = df.drop_duplicates(subset=['name', 'isin'])

    # TODO: Fill the other variables with the euronext values
    # TODO: Fill the 'symbol' value in the euronext with the bourso value

    return df

In [131]:
df.head()

,last,volume,symbol,name,date,boursorama,prefix,price,pea,currency,market,open,ticker,high,close,time_zone,low,last_trade_time,isin,turnover
1rPAC,37.29,405421,AC,ACCOR,2019-02-28 13:32:01.512834,1rPAC,1rP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FR0000120404,NaN
1rPACNV,46.02,0,ACNV,ACCOR,2019-02-28 13:32:01.512834,1rPACNV,1rP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FR0000120404,NaN
1rPADP,171.7,22674,ADP,ADP,2019-02-28 13:32:01.512834,1rPADP,1rP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FR0010340141,NaN
1rPAI,109.65,163127,AI,AIR LIQUIDE,2019-02-28 13:32:01.512834,1rPAI,1rP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FR0000120073,NaN
1rPAIR,113.24,285724,AIR,AIRBUS,2019-02-28 13:32:01.512834,1rPAIR,1rP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NL0000235190,NaN


| **Table**     | **Colonne**   | **Type**           | **Description** |
|--------------|-------------|------------------|--------------|
| **companies** | id        | SMALLINT (PK)   | Identifiant unique de l'entreprise |
|             | name        | VARCHAR         | Nom de l'entreprise |
|             | mid        | SMALLINT        | Identifiant du marché (référence vers `markets.id`) |
|             | symbol     | VARCHAR         | Symbole boursier de l'entreprise |
|             | isin       | CHAR(12)        | Code ISIN (International Securities Identification Number) |
|             | boursorama | VARCHAR         | Identifiant de l'entreprise sur Boursorama |
|             | euronext   | VARCHAR         | Identifiant de l'entreprise sur Euronext |
|             | pea       | BOOLEAN         | Indique si l'action est éligible au Plan d'Épargne en Actions (PEA) |
|             | sector1   | VARCHAR         | Secteur principal de l'entreprise |
|             | sector2   | VARCHAR         | Secteur secondaire de l'entreprise |
|             | sector3   | VARCHAR         | Secteur tertiaire de l'entreprise |


In [138]:
def populate_companies(df, db, df_markets):
    df_companies = pd.DataFrame()

    df_companies["isin"] = df["isin"].values
    df_companies["name"] = df["name"].values

    # Merge df with df_markets on the 'prefix' and 'boursorama' columns, with explicit suffixes
    merged_df = df.merge(
        df_markets[['boursorama', 'id', 'name']],
        how='left',
        left_on='prefix',
        right_on='boursorama',
        suffixes=('_action', '_market')  # Explicit suffixes to avoid conflicts
    )

    # Extract the 'id' and 'name_market' columns from the merged dataframe
    df_companies["mid"] = merged_df["id"].fillna(-1).astype(int)  # Ensure 'mid' is an integer
    df_companies["symbol"] = df["symbol"].values
    df_companies["boursorama"] = df["boursorama"].values
    df_companies["id"] = np.arange(len(df_companies))

    df_companies["euronext"] = df["ticker"].values
    eligible_pea = ["Bourse de Milan", "Mercados Espanoles", "Amsterdam", "Paris", "Deutsche Borse", "Bruxelle"]

    # Use the 'name_market' column to determine eligibility for PEA
    df_companies["pea"] = merged_df["name_market"].apply(lambda name: name in eligible_pea if pd.notna(name) else False)

    df_companies["sector1"] = ""  # TODO
    df_companies["sector2"] = ""  # TODO
    df_companies["sector3"] = ""  # TODO

    # Drop duplicates based on all columns except 'id'
    df_companies = df_companies.loc[:, ~df_companies.columns.isin(['id'])].drop_duplicates().reset_index(drop=True)

    # Reassign unique IDs after dropping duplicates
    df_companies["id"] = np.arange(len(df_companies))

    # db.df_write(df_companies, 'companies')
    return df_companies

Voici un tableau explicatif des colonnes de chaque table :  

| **Table**     | **Colonne**   | **Type**           | **Description** |
|--------------|-------------|------------------|--------------|
| **markets** | id          | SMALLINT (PK)   | Identifiant unique du marché |
|             | name        | VARCHAR         | Nom du marché |
|             | alias       | VARCHAR         | Alias du marché |
|             | boursorama  | VARCHAR         | Préfixe du marché sur Boursorama |
|             | sws        | VARCHAR         | Nom du marché sur Simply Wall Street |
|             | euronext    | VARCHAR         | Nom du marché sur Euronext |

In [9]:
from timescaledb_model import initial_markets_data

def populate_markets(df: pd.DataFrame, db, df_bourso: pd.DataFrame, df_eronext: pd.DataFrame):
    # Convert initial_markets_data to a DataFrame for easier manipulation
    initial_data = pd.DataFrame(
        initial_markets_data,
        columns=["id", "name", "alias", "boursorama", "euronext", "sws"]
    )

    # Create the markets DataFrame
    df_markets = pd.DataFrame()
    df_markets["id"] = initial_data["id"]
    df_markets["name"] = initial_data["name"]
    df_markets["alias"] = initial_data["alias"]

    # Map boursorama prefixes to the corresponding markets
    df_markets["boursorama"] = initial_data["boursorama"]

    # Map euronext tickers to the corresponding markets
    df_markets["euronext"] = np.nan

    # Fill the "sws" column with data from initial_markets_data
    df_markets["sws"] = initial_data["sws"]

    # Write the populated DataFrame to the database
    # db.df_write(df_markets, "markets")

    return df_markets

Voici un tableau explicatif des colonnes de chaque table :  

| **Table**     | **Colonne**   | **Type**           | **Description** |
|--------------|-------------|------------------|--------------|
| **stocks**  | date      | TIMESTAMPTZ     | Date et heure de la cotation |
|             | cid       | SMALLINT        | Identifiant de l'entreprise (référence vers `companies.id`) |
|             | value     | FLOAT4          | Valeur de l'action à cet instant |
|             | volume    | FLOAT4          | Volume échangé à cet instant |


In [145]:
def populate_stocks(df_boursorama: pd.DataFrame, db, df_companies: pd.DataFrame):
    df_stocks = pd.DataFrame()

    # Reset index to avoid ambiguity with 'symbol', without adding the old index as a column
    df_boursorama = df_boursorama.reset_index(drop=True)
    df_companies = df_companies.reset_index(drop=True)

    # Merge df_boursorama with df_companies on the 'symbol' column
    merged_df = df_boursorama.merge(
        df_companies[['symbol', 'id']],
        how='left',
        left_on='symbol',
        right_on='symbol',
        suffixes=('', '_company')  # Avoid suffix conflicts
    )

    # Populate the stocks dataframe
    df_stocks["date"] = merged_df["date"]
    df_stocks["cid"] = merged_df["id"].fillna(-1).astype(int)  # Ensure 'cid' is an integer
    df_stocks["value"] = merged_df["last"]
    df_stocks["volume"] = merged_df["volume"]

    # db.df_write(df_stocks, "stocks")
    return df_stocks

In [148]:
def populate_daystocks(df_euronext: pd.DataFrame, db, df_companies: pd.DataFrame):
    df_daystocks = pd.DataFrame()

    # Merge df_euronext with df_companies on the 'isin' column
    merged_df = df_euronext.merge(
        df_companies[['isin', 'id']],
        how='left',
        left_on='isin',
        right_on='isin',
        suffixes=('', '_company')  # Avoid suffix conflicts
    )

    # Populate the daystocks dataframe
    df_daystocks["date"] = merged_df["date"]
    df_daystocks["cid"] = merged_df["id"].fillna(-1).astype(int)  # Ensure 'cid' is an integer
    df_daystocks["open"] = merged_df["open"]
    df_daystocks["close"] = merged_df["close"]
    df_daystocks["high"] = merged_df["high"]
    df_daystocks["low"] = merged_df["low"]
    df_daystocks["volume"] = merged_df["volume"]
    df_daystocks["mean"] = (merged_df["high"] + merged_df["low"]) / 2
    df_daystocks["std"] = merged_df["high"] - merged_df["low"]

    # db.df_write(df_daystocks, "daystocks")
    return df_daystocks

In [114]:
df_markets = populate_markets(None, None, None, None)
df_markets.head(10)

,id,name,alias,boursorama,euronext,sws
0,1,New York,nyse,,NaN,
1,2,London Stock Exchange,lse,1u*.L,NaN,
2,3,Bourse de Milan,milano,1g,NaN,
3,4,Mercados Espanoles,mercados,FF55-,NaN,
4,5,Amsterdam,amsterdam,1rA,NaN,Amsterdam
5,6,Paris,paris,1rP,NaN,Paris
6,7,Deutsche Borse,xetra,1z,NaN,
7,8,Bruxelle,bruxelle,FF11_,NaN,Brussels
8,9,Australie,asx,,NaN,
9,100,International,int,,NaN,


In [139]:
df_companies = populate_companies(df, None, df_markets)
df_companies.head()

,isin,name,mid,symbol,boursorama,euronext,pea,sector1,sector2,sector3,id
0,FR0000120404,ACCOR,6,AC,1rPAC,NaN,True,,,,0
1,FR0000120404,ACCOR,6,ACNV,1rPACNV,NaN,True,,,,1
2,FR0010340141,ADP,6,ADP,1rPADP,NaN,True,,,,2
3,FR0000120073,AIR LIQUIDE,6,AI,1rPAI,NaN,True,,,,3
4,NL0000235190,AIRBUS,6,AIR,1rPAIR,NaN,True,,,,4


In [156]:
df_daystocks = populate_daystocks(df_euronext, None, df_companies)
df_daystocks.head()

,date,cid,open,close,high,low,volume,mean,std
0,2022-02-18,99,36.75,38.20,36.85,35.35,21155,36.100,1.50
1,2022-02-18,128,36.75,38.20,36.85,35.35,21155,36.100,1.50
2,2022-03-31,99,38.20,NaN,38.25,36.85,32389,37.550,1.40
3,2022-03-31,128,38.20,NaN,38.25,36.85,32389,37.550,1.40
4,2022-02-18,129,109.75,93.04,110.05,107.50,146619,108.775,2.55


In [144]:
df_stocks = populate_stocks(df_boursorama[:10], None, df_companies)
df_stocks.head(10)

,date,cid,value,volume
0,2019-02-28 13:32:01.512834,<NA>,58.01,0
1,2019-02-28 13:32:01.512834,0,37.29,405421
2,2019-02-28 13:32:01.512834,1,46.02,0
3,2019-02-28 13:32:01.512834,2,171.7,22674
4,2019-02-28 13:32:01.512834,<NA>,10.86,3512926
5,2019-02-28 13:32:01.512834,3,109.65,163127
6,2019-02-28 13:32:01.512834,4,113.24,285724
7,2019-02-28 13:32:01.512834,5,54.8,2613
8,2019-02-28 13:32:01.512834,<NA>,3.46,0
9,2019-02-28 13:32:01.512834,<NA>,3.5,0


In [140]:
df_companies[127:128]

,isin,name,mid,symbol,boursorama,euronext,pea,sector1,sector2,sector3,id
127,FR0010313833,ARKEMA,6,AKENV,1rPAKENV,NaN,True,,,,127


In [97]:
df_companies.head(3)

,isin,name,mid,symbol,boursorama,id,euronext,pea,sector1,sector2,sector3
0,FR0000120404,ACCOR,6,AC,1rPAC,0,NaN,True,,,
1,FR0000120404,ACCOR,6,ACNV,1rPACNV,1,NaN,True,,,
2,FR0010340141,ADP,6,ADP,1rPADP,2,NaN,True,,,


In [93]:
len(df_stocks1)

16

In [85]:
len(df_companies)

1167